In [11]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

In [12]:
# 0) 관련 모듈 임포트 
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

### 셀레니움 + 뷰티풀숩
```
1) 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동
2) 검색어 입력 
: input() 함수를 이용해서 검색어를 입력받능 후 다나와의 검색창에 연결.
현재 html 페이지를 변수에 저장
3) 페이지 소스 저장 및 뷰티풀숩 객체화 
4) 검색된 결과 페이지에서 제품 리스트 추출 
5) 1개의 상품만 출력
6) 1페이지 전체 => 2차원 리스트 => 데이타프레임으로 변경 및 csv 저장
```
### 다나와
https://www.danawa.com
### 검색어 입력후 1페이지에서 제품 정보 데이터 추출하기
- 검색어는 입력문(input) 이용
- 평점과 스펙이 없는 경우에는 '' 처리

In [3]:
# 입력문(input) 연습 
word = input('검색어를 입력해주세요...')
print('입력한 검색어는 ', word, '입니다')

검색어를 입력해주세요...삼성전자 갤럭시 워치 액티브2
입력한 검색어는  삼성전자 갤럭시 워치 액티브2 입니다


In [4]:
# 1) 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동 

driver = webdriver.Chrome('chromedriver.exe')
time.sleep(2)
url = 'https://www.danawa.com'
driver.get(url)

In [5]:
### 검색어 입력후 1페이지에서 제품 정보 데이타 추출하기
#2) 검색어 입력 
#: input() 함수를 이용해서 검색어를 입력받능 후 다나와의 검색창에 연결. 현재 html 페이지를 변수에 저장
# k1는 입력창의 name 값 
word = input('검색어 입력하기 =>')
#다나와의 검색창 요소 접근
target = driver.find_element_by_name('k1')
# 입력받은 검색어를 검색창에 입력
target.send_keys(word)
time.sleep(1)
# 전송 및 결과 출력
target.submit()

검색어 입력하기 =>삼성전자 갤럭시 워치 액티브2


In [6]:
# 3) 페이지 소스 저장 및 뷰티풀숩 객체화 
html_str = driver.page_source
soup = BeautifulSoup(html_str)
soup

<html class="js no-touch" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>삼성전자 갤럭시 워치 액티브2 : 다나와 통합검색</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="삼성전자 갤럭시 워치 액티브2 통합검색 : 다나와 통합검색" name="title"/>
<meta content="'삼성전자 갤럭시 워치 액티브2'의 다나와 통합검색 결과입니다." name="description"/>
<meta content="삼성전자 갤럭시 워치 액티브2, 다나와, 최저가, 가격비교, 검색, 통합검색, 상품 추천, 가격비교사이트, 가격비교 사이트, 가격비교 싸이트, 가격 검색, 추천, 인터넷쇼핑, 온라인쇼핑, 쇼핑, 쇼핑몰, 싸게 파는 곳, 지식쇼핑" name="keywords"/>
<meta content="삼성전자 갤럭시 워치 액티브2 통합검색 : 다나와 통합검색" property="og:title"/>
<meta content="'삼성전자 갤럭시 워치 액티브2'의 다나와 통합검색 결과입니다." property="og:description"/>
<meta content="https://img.danawa.com/new/tour/img/logo/sns_danawa.jpg" property="og:image"/>
<meta content="width=1280" name="viewport"/>
<link href="//img.danawa.com/new/danawa_main/v1/img/danawa_favicon.ico" rel="shortcut icon"/>
<link href="//static.danawa.com/css/common.css?v=220222093

In [7]:
# 4) 검색된 결과 페이지에서 제품 리스트 추출
# target_list = soup.select('ul.product_list > li')
# len(target_list)
# target_list = soup.select('ul.product_list > li.prod_item')
target_list = soup.select('div.prod_main_info')
len(target_list)

41

In [8]:
# 5) 1개의 상품만 출력
# 제품명, 링크, 가격, 평정, 등록일, 스펙
prod_name = target_list[0].select_one('p.prod_name a').text
prod_url = target_list[0].select_one('p.prod_name a')['href']
price = target_list[0].select_one('p.price_sect a').text

# 평점이 없다면 공백처리
# 평점이 있다면 데이타 추출
if target_list[0].select_one('div.point_num strong'):
    score = target_list[0].select_one('div.point_num strong').text
else:
    score = ''

reg_date = target_list[0].select_one('dl.meta_item.mt_date dd').text
    
# 스펙이 없다면 공백처리
# 스펙이 있다면 데이타 추출
if target_list[0].select_one('div.spec_list'):
    spec= target_list[0].select_one('div.spec_list').text.strip().replace('\n','').replace('\t', '')
else:
    spec = ''
    
print('제품명 :', prod_name)
print('링크 :', prod_url)
print('가격 :', price)
print('평점 :', score)
print('등록일 :', reg_date)
print('스펙 :', spec)

# target_list[1]

제품명 : 삼성전자 갤럭시 워치 액티브2 40mm 알루미늄
링크 : http://prod.danawa.com/info/?pcode=8964230&keyword=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90+%EA%B0%A4%EB%9F%AD%EC%8B%9C+%EC%9B%8C%EC%B9%98+%EC%95%A1%ED%8B%B0%EB%B8%8C2&cate=12215657
가격 : 210,650원
평점 : 4.8
등록일 : 2019.09.
스펙 : 스마트워치 / SAMOLED / 360x360 / 3.02cm(1.2인치) / 26g / [사양] 타이젠OS / 삼성 엑시노스9110 / 듀얼 코어 / 1.15GHz / 램: 768MB / 내장메모리: 4GB / [운동] 운동량 / 속도 / 이동거리 / 페이스 / 활동시간 / 보행수 / 걷기 / 달리기 / 날씨 / 로잉머신 / 골프 / 일립티컬 / 수영 / 자전거타기 / 요가 / 고도 / 등산 / [건강] 심전도 / 혈압 / 심박수 측정 / 칼로리 / 수면 정보 / 여성 건강 모니터링 / [연결] WiFi / GPS / NFC / 블루투스 / v5.0 / [기능] 스피커 / 마이크 / SNS알림 / 음악콘트롤 / 메세지알림 / 무선충전 / 전화알림 / 통화가능 / 진동 / AI음성인식 / 방수·방진: 5ATM, IP68 / [배터리] 용량:247mAh / 평균 사용:3일 이하 / MIL-STD-810G / AOD


In [9]:
# 6) 1페이지 전체 => 2차원 리스트 => 데이타프레임으로 변경 및 csv 저장

result_list = []
for target in target_list:
    prod_name = target.select_one('p.prod_name a').text
    prod_url = target.select_one('p.prod_name a')['href']
    price = target.select_one('p.price_sect a').text

    # 평점이 없다면 공백처리 
    # 평점이 있다면 데이타 추출 
    if target.select_one('div.point_num strong'): 
        score = target.select_one('div.point_num strong').text
    else:
        score = ''
    
    if target.select_one('dl.meta_item.mt_date dd'):
        reg_date = target.select_one('dl.meta_item.mt_date dd').text
    else:
        reg_date = ''

    # 스펙이 없다면 공백처리 
    # 스펙이 있다면 데이타 추출 
    if target.select_one('div.spec_list'): 
        spec = target.select_one('div.spec_list').text.strip().replace('\n','').replace('\t', '')
    else:
        spec = ''
    
    result_list.append([prod_name, prod_url, price, score, reg_date, spec])

df = pd.DataFrame(result_list, columns=['제품명', '링크', '가격', '평점', '등록일', '스펙'])
df

AttributeError: 'NoneType' object has no attribute 'text'

### 페이지 클릭 테스트

In [15]:
# 페이징 부분 요소 추출
target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')

In [ ]:
# 2번째 페이지 클릭
target_page[1].click()

In [16]:
# 3번째 페이지 클릭
target_page[2].click()

### 10페이지 검색 데이타

### 작업 과정
```
1) 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동
2) 검색어 입력 
: input() 함수를 이용해서 검색어를 입력받능 후 다나와의 검색창에 연결.
현재 html 페이지를 변수에 저장
3) 페이지 소스 저장 및 뷰티풀숩 객체화 
4) 검색된 결과 페이지에서 제품 리스트 추출 
5) 1페이지 전체 => 2차원 리스트
6) 10페이지의 하이퍼링크 요소추출
2번째 페이지 클릭 후 3~5번 과정 반복
7) 2~10페이지 버튼 클릭해서 리스트에 데이타 추가
8) 데이타프레임으로 변경 및 csv 저장
```

In [20]:
# 1) 셀레니움으로 크롬웹브라우저 실행, 다나와 웹페이지 이동 

driver = webdriver.Chrome('chromedriver.exe')
time.sleep(2)
url = 'https://www.danawa.com'
driver.get(url)
time.sleep(1)
# 2) 검색어 입력 
#: input() 함수를 이용해서 검색어를 입력받능 후 다나와의 검색창에 연결. 현재 html 페이지를 변수에 저장
# k1는 입력창의 name 값 
word = input('검색어 입력하기 =>')
#다나와의 검색창 요소 접근
target = driver.find_element_by_name('k1')
# 입력받은 검색어를 검색창에 입력
target.send_keys(word)
time.sleep(1)
# 전송 및 결과 출력
target.submit()
time.sleep(1)
# 3) 페이지 소스 저장 및 뷰티풀숩 객체화 
html_str = driver.page_source
soup = BeautifulSoup(html_str)
time.sleep(1)
# 4) 검색된 결과 페이지에서 제품 리스트 추출
target_list = soup.select('div.prod_main_info')
len(target_list)

# 6) 1페이지 전체 => 2차원 리스트 => 데이타프레임으로 변경 및 csv 저장
time.sleep(1)
result_list = []
for target in target_list:
    prod_name = target.select_one('p.prod_name a').text
    prod_url = target.select_one('p.prod_name a')['href']
    price = target.select_one('p.price_sect a').text

    # 평점이 없다면 공백처리 
    # 평점이 있다면 데이타 추출 
    if target.select_one('div.point_num strong'): 
        score = target.select_one('div.point_num strong').text
    else:
        score = ''
    
    if target.select_one('dl.meta_item.mt_date dd'):
        reg_date = target.select_one('dl.meta_item.mt_date dd').text
    else:
        reg_date = ''

    # 스펙이 없다면 공백처리 
    # 스펙이 있다면 데이타 추출 
    if target.select_one('div.spec_list'): 
        spec = target.select_one('div.spec_list').text.strip().replace('\n','').replace('\t', '')
    else:
        spec = ''
    
    result_list.append([prod_name, prod_url, price, score, reg_date, spec])

검색어 입력하기 =>로봇 청소기


In [21]:
# 6) 10페이지의 하이퍼링크 요소추출
target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')

In [22]:
# 7) 2~10페이지 버튼 클릭해서 리스트에 데이타 추가
for i in range(1, 3):
    target_page = driver.find_elements_by_css_selector('div.paging_number_wrap a')
    time.sleep(2)
    # 페이지 이동
    target_page[i].click()
    print(i+1,'번째 페이지')
    # 3) 페이지 소스 저장 및 뷰티풀숩 객체화 
    html_str = driver.page_source
    soup = BeautifulSoup(html_str)
    time.sleep(1)
    # 4) 검색된 결과 페이지에서 제품 리스트 추출
    target_list = soup.select('div.prod_main_info')
    len(target_list)
    for target in target_list:
        prod_name = target.select_one('p.prod_name a').text
        prod_url = target.select_one('p.prod_name a')['href']
        price = target.select_one('p.price_sect a').text
    
        # 평점이 없다면 공백처리 
        # 평점이 있다면 데이타 추출 
        if target.select_one('div.point_num strong'): 
            score = target.select_one('div.point_num strong').text
        else:
            score = ''
        
        if target.select_one('dl.meta_item.mt_date dd'):
            reg_date = target.select_one('dl.meta_item.mt_date dd').text
        else:
            reg_date = ''
    
        # 스펙이 없다면 공백처리 
        # 스펙이 있다면 데이타 추출 
        if target.select_one('div.spec_list'): 
            spec = target.select_one('div.spec_list').text.strip().replace('\n','').replace('\t', '')
        else:
            spec = ''
        
        result_list.append([prod_name, prod_url, price, score, reg_date, spec])

2 번째 페이지
3 번째 페이지


In [23]:
# 8) 데이타프레임으로 변경 및 csv 저장
df = pd.DataFrame(result_list, columns=['제품명', '링크', '가격', '평점', '등록일', '스펙'])
df

,제품명,링크,가격,평점,등록일,스펙
0,씽크웨이 ThinkAir RV50 PRO,http://prod.danawa.com/info/?pcode=16990496&ke...,"648,780원",,2022.05.,로봇청소기 / 흡입+걸레겸용 / [센서] LDS라이다센서 / 자이로센서 / ...
1,엠지텍 트윈보스 S9 PRO MASTER,http://prod.danawa.com/info/?pcode=13943729&ke...,"598,990원",,2021.04.,로봇청소기 / 흡입+걸레겸용 / [센서] ToF센서 / LDS라이다센서 / ...
2,에코백스 디봇 오즈모 T9 (본체+오토엠티스테이션),http://prod.danawa.com/info/?pcode=13863530&ke...,"598,990원",,2021.04.,로봇청소기 / 흡입+걸레겸용 / [센서] ToF센서 / 장애물인식센서 / 추...
3,로보락 S7 MaxV Ultra,http://prod.danawa.com/info/?pcode=16613441&ke...,"2,049,720원",4.9,2022.04.,로봇청소기 / 흡입+걸레겸용 / [센서] 카메라2개 / LDS라이다센서 / 3D센서...
4,삼성전자 비스포크 제트 봇 VR30T80313,http://prod.danawa.com/info/?pcode=14056946&ke...,"341,500원",4.7,2021.04.,로봇청소기 / 흡입전용 / [센서] LDS라이다센서 / 추락방지센서 / [성능] 인...
...,...,...,...,...,...,...
121,바이마르 VMK-ROB2021M,http://prod.danawa.com/info/?pcode=14628983&ke...,"28,900원",,2021.06.,로봇청소기 / 흡입+걸레겸용 / [성능] 사용시간(개당): 1시간30분 / [부가]...
122,로엘 듀스핀로봇,http://prod.danawa.com/info/?pcode=12243629&ke...,"79,000원",4.0,2020.09.,로봇청소기 / 걸레전용 / [성능] 사용시간(개당): 1시간40분 / 충전시간: 4...
123,치후360 S10,http://prod.danawa.com/info/?pcode=14729138&ke...,"498,800원",4.7,2021.07.,로봇청소기 / 흡입+걸레겸용 / [센서] ToF센서 / 3D센서 / 장애물인식센서 ...
124,LG전자 코드제로 로보킹 ThinQ R7 R76STM,http://prod.danawa.com/info/?pcode=10553172&ke...,"477,000원",4.7,2020.02.,로봇청소기 / 흡입+걸레겸용 / [센서] 카메라2개 / 초음파센서 / [성능] 인버...


In [25]:
df.to_csv('output/다나와 로봇청소기.csv', index = False)